In [1]:
using Pkg
Pkg.activate(".")
using Statistics
using LinearAlgebra
using PlotlyJS

  Activating project at `c:\Users\uiv02757\OneDrive - Vitesco Technologies\git_repos\preconditioning`
┌ Warning: It looks like the Kaleido process is not responding. 
│ The unresponsive process will be killed, but this means that you will not be able to save figures using `savefig`.
│ 
│ If you are on Windows this might be caused by known problems with Kaleido v0.2 on Windows (you are using version 0.2.1).
│ You might want to try forcing a downgrade of the Kaleido_jll library to 0.1.
│ Check the Package Readme at https://github.com/JuliaPlots/PlotlyKaleido.jl/tree/main#windows-note for more details.
│ 
│ If you think this is not your case, you might try using a longer timeout to check if the process is not responding (defaults to 10 seconds) by passing the desired value in seconds using the `timeout` kwarg when calling `PlotlyKaleido.start` or `PlotlyKaleido.restart`
└ @ PlotlyKaleido C:\Users\uiv02757\.julia\packages\PlotlyKaleido\uMRVG\src\PlotlyKaleido.jl:24


In [2]:
S(a) = [
    [a[1] 0. 0. ];
    [0. a[2] 0. ];
    [0. 0. a[3] ]
]
R₁(α) = [
    [cos(α) -sin(α) 0.];
    [sin(α)  cos(α) 0.];
    [0. 0. 1.]
]
R₂(α) = [
    [ cos(α) 0 sin(α)];
    [0 1 0];
    [-sin(α) 0 cos(α)]
]

R₂ (generic function with 1 method)

In [3]:
n = 20
δ = range(-Base.π, Base.π, length=n)
λ = range(-Base.π, Base.π, length=n)

y₁ = [x * y for (x, y) in Iterators.product(cos.(δ), sin.(δ))]
y₂ = [x * y for (x, y) in Iterators.product(sin.(δ), sin.(λ))]
y₃ = [x * y for (x, y) in Iterators.product(ones(length(δ)), cos.(δ))]

Y₁ = reshape(y₁,:); Y₂ = reshape(y₂,:); Y₃ = reshape(y₃,:)
Y = [Y₁'; Y₂'; Y₃']

a = [1, 2, 3]
c = [2, 2, 2]
ψ = Base.π/9; θ = Base.π/6; ϕ = Base.π\3

F(x) = S(1 ./a) * R₁(ψ) * R₂(θ) * R₁(ϕ)' * (x .-c)
inv_F(y) = c .+ R₁(ϕ) * R₂(θ)' * R₁(ψ)' * S(a) * y

X = inv_F(Y)
X₁ = X[1,:]; X₂ = X[2,:]; X₃ = X[3,:]

x₁ = reshape(X₁, (n,n)); x₂ = reshape(X₂, (n,n)); x₃ = reshape(X₃, (n,n))

fig = make_subplots(rows=1, cols=2,  specs=fill(Spec(kind="scene"), 1, 2)) 

s₁ = surface(x = y₁, y = y₂, z = y₃)
s₂ = surface(x = x₁, y = x₂, z = x₃)

add_trace!(fig, s₁, row=1, col=1)
add_trace!(fig, s₂, row=1, col=2)

fig

┌ Warning: It looks like the Kaleido process is not responding. 
│ The unresponsive process will be killed, but this means that you will not be able to save figures using `savefig`.
│ 
│ If you are on Windows this might be caused by known problems with Kaleido v0.2 on Windows (you are using version 0.2.1).
│ You might want to try forcing a downgrade of the Kaleido_jll library to 0.1.
│ Check the Package Readme at https://github.com/JuliaPlots/PlotlyKaleido.jl/tree/main#windows-note for more details.
│ 
│ If you think this is not your case, you might try using a longer timeout to check if the process is not responding (defaults to 10 seconds) by passing the desired value in seconds using the `timeout` kwarg when calling `PlotlyKaleido.start` or `PlotlyKaleido.restart`
└ @ PlotlyKaleido C:\Users\uiv02757\.julia\packages\PlotlyKaleido\uMRVG\src\PlotlyKaleido.jl:24
┌ Warning: It looks like the Kaleido process is not responding. 
│ The unresponsive process will be killed, but this means tha

data: [
  "surface with fields scene, type, x, y, and z",
  "surface with fields scene, type, x, y, and z"
]

layout: "layout with fields annotations, margin, scene, scene2, and template"

In [6]:
# Traduction of the matlab code : https://fr.mathworks.com/matlabcentral/fileexchange/24693-ellipsoid-fit
function fit_ellipse(x, y, z)
    # quadratic form of ellipse
    D = hcat( 
        x .* x + y .* y - 2 .* z .* z,
        x .* x + z .* z - 2 .* y .* y,
        2 .* x .* y,
        2 .* x .* z,
        2 .* y .* z,
        2 .* x,
        2 .* y,
        2 .* z,
        1 .* ones(size(x)))            
    
    d2 = x .* x + y .* y + z .* z;      # the RHS of the llsq problem (y's)
    u = (D' * D)\(D' * d2);             # solution to the normal equations
    # find the residual sum of errors
    # chi2 = sum( ( 1 - ( D * u ) ./ d2 ).^2 ); % this chi2 is in the coordinate frame in which the ellipsoid is a unit sphere.

    # find the ellipsoid parameters
    # convert back to the conventional algebraic form
    v = zeros(10,)
    v[1] = u[1] + u[2] - 1;
    v[2] = u[1] - 2 * u[2] - 1;
    v[3] = u[2] - 2 * u[1] - 1;
    v[4:10] = u[3:9];
    
    # form the algebraic form of the ellipsoid
    A = [   v[1] v[4] v[5] v[7];
            v[4] v[2] v[6] v[8];
            v[5] v[6] v[3] v[9];
            v[7] v[8] v[9] v[10] 
        ];

    # find the center of the ellipsoid
    c = -A[1:3,1:3] \ v[7:9];
    T = Matrix{Base.Float64}(I, 4, 4)
    T[4,1:3] = c[1:3];
    
    # translate to the center
    R = T * A * transpose(T);

    # solve the eigenproblem
    M = R[1:3,1:3] / -R[4,4]

    evals = eigvals(M)
    evecs = eigvecs(M)

    radii = sqrt.( 1 ./ abs.(evals) )
    sgns = sign.(evals)
    radii = radii .* sgns

    M = evecs * S(radii)

    ## ADDED
    # α₁ = M[1,1]; α₂ = M[2,1]; α₃ = M[3,1]
    # δ₀ = atan(α₃, α₁)
    # α₁_bis = α₁* cos(δ₀) + α₃ * sin(δ₀)
    # λ₀ = atan(-α₂, α₁_bis)
    # M2 =  M * R₁(λ₀)' * R₂(δ₀)'
    ## END ADDED

    return [M, c, radii, evecs]
end

M, c, a, vecs = fit_ellipse(X₁, X₂, X₃)

X_fit = M * Y .+ c

X₁_fit = X_fit[1,:]
X₂_fit = X_fit[2,:]
X₃_fit = X_fit[3,:]

x₁_fit = reshape(X₁_fit, (n,n))
x₂_fit = reshape(X₂_fit, (n,n))
x₃_fit = reshape(X₃_fit, (n,n))

# println(fit_M)

fig = make_subplots(rows=1, cols=2,  specs=fill(Spec(kind="scene"), 1, 2)) 

# s₁ = surface(x = y₁_fit, y = y₂_fit, z = y₃_fit)
s₂ = surface(x = x₁_fit, y = x₂_fit, z = x₃_fit, opacity = 0.2)
s₃ = scatter(x = X₁, y = X₂, z = X₃, 
    mode = "markers", type = "scatter3d",
    name = "Datas",
    marker=attr(
        size=2,
        opacity=0.8
    )        
)
s₄ = scatter( 
    x = [c[1], c[1] + a[1] * vecs[1,1],
         c[1], c[1] + a[2] * vecs[1,2],
         c[1], c[1] + a[3] * vecs[1,3]],
    y = [c[2], c[2] + a[1] * vecs[2,1],
         c[2], c[2] + a[2] * vecs[2,2],
         c[2], c[2] + a[3] * vecs[2,3]],
    z = [c[3], c[3] + a[1] * vecs[3,1], 
         c[3], c[3] + a[2] * vecs[3,2],
         c[3], c[3] + a[3] * vecs[3,3]], 
    mode = "lines + markers", type = "scatter3d",
    name = "Datas",
    marker=attr(
        size=4,
        opacity=0.8
    )        
)

add_trace!(fig, s₂, row=1, col=1)
add_trace!(fig, s₃, row=1, col=1)
add_trace!(fig, s₄, row=1, col=1)

fig

┌ Warning: It looks like the Kaleido process is not responding. 
│ The unresponsive process will be killed, but this means that you will not be able to save figures using `savefig`.
│ 
│ If you are on Windows this might be caused by known problems with Kaleido v0.2 on Windows (you are using version 0.2.1).
│ You might want to try forcing a downgrade of the Kaleido_jll library to 0.1.
│ Check the Package Readme at https://github.com/JuliaPlots/PlotlyKaleido.jl/tree/main#windows-note for more details.
│ 
│ If you think this is not your case, you might try using a longer timeout to check if the process is not responding (defaults to 10 seconds) by passing the desired value in seconds using the `timeout` kwarg when calling `PlotlyKaleido.start` or `PlotlyKaleido.restart`
└ @ PlotlyKaleido C:\Users\uiv02757\.julia\packages\PlotlyKaleido\uMRVG\src\PlotlyKaleido.jl:24
┌ Warning: It looks like the Kaleido process is not responding. 
│ The unresponsive process will be killed, but this means tha

data: [
  "surface with fields opacity, scene, type, x, y, and z",
  "scatter3d with fields marker, mode, name, scene, type, x, y, and z",
  "scatter3d with fields marker, mode, name, scene, type, x, y, and z"
]

layout: "layout with fields annotations, margin, scene, scene2, and template"